In [1]:
import repo_chat.git2vectors as git2vectors
from repo_chat.chat_utils import RetrievalChain
from IPython.display import Markdown
import pandas as pd
import json

c:\Users\voyno\Desktop\code\repos\repo-chat\venv\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Create vectorstore, this will take a while
repo = "https://github.com/smol-ai/developer"
git2vectors.create_vectorstore(repo)

# load vectorstore, this is fast
vectorstore = git2vectors.get_vectorstore()

In [3]:
# Create an instance of the class
chain = RetrievalChain(vectorstore, repo)

# Let's say we have a query
query = "Explain this to me like I\'m 5."

# generic retrieval query
response = chain.chat(query)
Markdown(response['text'])

This github repository contains a prototype for a "junior developer" agent that scaffolds an entire codebase based on a product spec. It is designed to be helpful, harmless, and honest, with a codebase that is simple, safe, and small. The program uses prompts to generate code, and the user can manually run and identify errors, or paste the error into the prompt to file a github issue. The program is a Chrome extension that summarizes web pages using the Anthropic Claude API, and uses a combination of JavaScript, HTML, and CSS to provide a user-friendly interface. The repository also includes a video demo and an architecture diagram.